<a href="https://colab.research.google.com/github/ryybiinana1/HW_ML_NLP/blob/main/hw_language_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Домашнее задание. Обучение языковой модели с помощью LSTM (10 баллов)

Э
В этом задании Вам предстоит обучить языковую модель с помощью рекуррентной нейронной сети. В отличие от семинарского занятия, Вам необходимо будет работать с отдельными словами, а не буквами.


Установим модуль ```datasets```, чтобы нам проще было работать с данными.

In [1]:
!pip install datasets

Импорт необходимых библиотек

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List

import seaborn
seaborn.set(palette='summer')

In [3]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Воспользуемся датасетом imdb. В нем хранятся отзывы о фильмах с сайта imdb. Загрузим данные с помощью функции ```load_dataset```

In [5]:
# Загрузим датасет
dataset = load_dataset('imdb')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

### Препроцессинг данных и создание словаря (1 балл)

Далее вам необходмо самостоятельно произвести препроцессинг данных и получить словарь или же просто ```set``` строк. Что необходимо сделать:

1. Разделить отдельные тренировочные примеры на отдельные предложения с помощью функции ```sent_tokenize``` из бибилиотеки ```nltk```. Каждое отдельное предложение будет одним тренировочным примером.
2. Оставить только те предложения, в которых меньше ```word_threshold``` слов.
3. Посчитать частоту вхождения каждого слова в оставшихся предложениях. Для деления предлоения на отдельные слова удобно использовать функцию ```word_tokenize```.
4. Создать объект ```vocab``` класса ```set```, положить в него служебные токены '\<unk\>', '\<bos\>', '\<eos\>', '\<pad\>' и vocab_size самых частовстречающихся слов.   

In [118]:
sentences = []
word_threshold = 32

for document in tqdm(dataset['train']['text']):
  tokenize_document = sent_tokenize(document, language='english')

  for sentence in tokenize_document:
    if len(word_tokenize(sentence)) >= word_threshold:
      sentences.append(sentence.lower())
# Получить отдельные предложения и поместить их в sentences

  0%|          | 0/25000 [00:00<?, ?it/s]

In [99]:
print("Всего предложений:", len(sentences))
print(dataset["train"]["text"][0])
print(sentences[0])

Всего предложений: 72256
I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are

Посчитаем для каждого слова его встречаемость.

In [119]:
words = Counter()

for sentence in tqdm(sentences):
  for word in word_tokenize(sentence):
    words[word] += 1

  0%|          | 0/72256 [00:00<?, ?it/s]

In [101]:
print(len(words))

87302


Добавим в словарь ```vocab_size``` самых встречающихся слов.

In [147]:
vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
vocab_size = 20000

print(f"Начальный размер vocab: {len(vocab)}")
for word, cnt in sorted(words.items(), key=lambda x: x[1], reverse=True):
  if len(vocab) >= (vocab_size + 4):
    break
  vocab.add(word)

print(f"Финальный размер vocab: {len(vocab)}")

Начальный размер vocab: 4
Финальный размер vocab: 20004


In [141]:
assert '<unk>' in vocab
assert '<bos>' in vocab
assert '<eos>' in vocab
assert '<pad>' in vocab
assert len(vocab) == vocab_size + 4

In [142]:
print("Всего слов в словаре:", len(vocab))

Всего слов в словаре: 10004


### Подготовка датасета (1 балл)

Далее, как и в семинарском занятии, подготовим датасеты и даталоадеры.

В классе ```WordDataset``` вам необходимо реализовать метод ```__getitem__```, который будет возвращать сэмпл данных по входному idx, то есть список целых чисел (индексов слов).

Внутри этого метода необходимо добавить служебные токены начала и конца последовательности, а также токенизировать соответствующее предложение с помощью ```word_tokenize``` и сопоставить ему индексы из ```word2ind```.

In [148]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [149]:
class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        words = word_tokenize(self.data[idx])
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [word2ind.get(word, self.unk_id) for word in words]
        tokenized_sentence += [self.eos_id]

        return tokenized_sentence

    def __len__(self) -> int:
        return len(self.data)

In [126]:
def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind['<pad>']) -> torch.Tensor:
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:,:-1],
        'target_ids': sequences[:,1:]
    }

    return new_batch

In [96]:
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2)
eval_sentences, test_sentences = train_test_split(sentences, test_size=0.5)

train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)
test_dataset = WordDataset(test_sentences)

batch_size = 64

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

**В коде выше неправильно проведено деление на train, eval, test.** Следствие: модель будет видеть одни и те же данные и в обучении, и в тестировании. Она "запоминит" ответы вместо того, чтобы обобщать

Сделаем другое разделение, и продолжим обучение на его основе.

In [127]:
train_eval_sentences, test_sentences = train_test_split(sentences, test_size=0.2)
train_sentences, eval_sentences = train_test_split(train_eval_sentences, test_size=0.25)

batch_size = 64

train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)
test_dataset = WordDataset(test_sentences)

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

## Обучение и архитектура модели

Вам необходимо на практике проверить, что влияет на качество языковых моделей. В этом задании нужно провести серию экспериментов с различными вариантами языковых моделей и сравнить различия в конечной перплексии на тестовом множестве.

Возмоэные идеи для экспериментов:

* Различные RNN-блоки, например, LSTM или GRU. Также можно добавить сразу несколько RNN блоков друг над другом с помощью аргумента num_layers. Вам поможет официальная документация [здесь](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
* Различные размеры скрытого состояния. Различное количество линейных слоев после RNN-блока. Различные функции активации.
* Добавление нормализаций в виде Dropout, BatchNorm или LayerNorm
* Различные аргументы для оптимизации, например, подбор оптимального learning rate или тип алгоритма оптимизации SGD, Adam, RMSProp и другие
* Любые другие идеи и подходы

После проведения экспериментов необходимо составить таблицу результатов, в которой описан каждый эксперимент и посчитана перплексия на тестовом множестве.

Учтите, что эксперименты, которые различаются, например, только размером скрытого состояния или количеством линейных слоев считаются, как один эксперимент.

Успехов!

### Функция evaluate (1 балл)

Заполните функцию ```evaluate```

In [128]:
def evaluate(model, criterion, dataloader) -> float:
    model.eval()
    perplexity = []
    with torch.no_grad():
        for batch in dataloader:
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(logits, batch['target_ids'].flatten())
            perplexity.append(torch.exp(loss).item())

    perplexity = sum(perplexity) / len(perplexity)

    return perplexity

### Train loop (1 балл)

Напишите функцию для обучения модели.

In [129]:
def train_model(train_dataloader, model, optimizer, criterion, num_epoch ):
  losses = []
  perplexities = []

  for epoch in range(num_epoch):
    epoch_losses = []
    model.train()
    for batch in tqdm(train_dataloader, desc= f'Training epoch {epoch}:'):
      optimizer.zero_grad()
      logits = model(batch['input_ids']).flatten(start_dim = 0, end_dim = 1)
      loss = criterion(logits, batch['target_ids'].flatten())
      loss.backward()
      optimizer.step()

      epoch_losses.append(loss.item())

    losses.append(sum(epoch_losses)/ len(epoch_losses))
    perplexities.append(evaluate(model, criterion))
  return losses, perplexities

### Первый эксперимент (2 балла)

Определите архитектуру модели и обучите её.

In [144]:
class LanguageModel(nn.Module):
    def __init__(self, hidden_dim: int, vocab_size: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, num_layers = 2)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p = 0.1)

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        embeddings = self.embedding(input_batch)
        output, _ = self.rnn(embeddings)
        output = self.dropout(self.linear(self.non_lin(output)))
        projection= self.projection(self.non_lin(output))

        return projection

In [152]:
model = LanguageModel(hidden_dim=64, vocab_size=len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters())

AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [145]:
print(train_model(train_dataloader, model, optimizer, criterion, 20))

Training epoch 0::   0%|          | 0/678 [00:00<?, ?it/s]

AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [146]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # ВКЛЮЧИТЕ ОТЛАДКУ!

# 1. Проверка словаря
print("=== ПРОВЕРКА СЛОВАРЯ ===")
print(f"Размер vocab: {len(vocab)}")
print(f"Диапазон индексов: {min(word2ind.values())} - {max(word2ind.values())}")

# 2. Проверка данных
print("\n=== ПРОВЕРКА ДАННЫХ ===")
def check_indices(dataset, name):
    all_indices = []
    for i in range(min(100, len(dataset))):  # Проверим первые 100 примеров
        sample = dataset[i]
        all_indices.extend(sample)

    max_idx = max(all_indices)
    min_idx = min(all_indices)
    print(f"{name}: min={min_idx}, max={max_idx}, vocab_size={len(vocab)}")

    if max_idx >= len(vocab):
        print(f"❌ ОШИБКА в {name}: индекс {max_idx} >= vocab_size {len(vocab)}")
    if min_idx < 0:
        print(f"❌ ОШИБКА в {name}: отрицательный индекс {min_idx}")
    return max_idx, min_idx

check_indices(train_dataset, "Train")
check_indices(eval_dataset, "Eval")

# 3. Тестовый прогон на CPU
print("\n=== ТЕСТ НА CPU ===")
device_cpu = torch.device('cpu')
model_cpu = LanguageModel(hidden_dim=64, vocab_size=len(vocab)).to(device_cpu)

# Протестируем один батч на CPU
for i, batch in enumerate(train_dataloader):
    if i >= 1:  # Только первый батч
        break

    print(f"Батч {i}:")
    print(f"  input_ids shape: {batch['input_ids'].shape}")
    print(f"  input_ids range: {batch['input_ids'].min()} - {batch['input_ids'].max()}")
    print(f"  target_ids range: {batch['target_ids'].min()} - {batch['target_ids'].max()}")

    # Перенос на CPU для теста
    input_cpu = batch['input_ids'].to(device_cpu)
    target_cpu = batch['target_ids'].to(device_cpu)

    try:
        with torch.no_grad():
            logits = model_cpu(input_cpu).flatten(0, 1)
            loss = criterion(logits, target_cpu.flatten())
        print(f"  ✅ Успех на CPU, loss: {loss.item()}")
    except Exception as e:
        print(f"  ❌ Ошибка на CPU: {e}")

=== ПРОВЕРКА СЛОВАРЯ ===
Размер vocab: 10004
Диапазон индексов: 0 - 20003

=== ПРОВЕРКА ДАННЫХ ===
Train: min=33, max=19995, vocab_size=10004
❌ ОШИБКА в Train: индекс 19995 >= vocab_size 10004
Eval: min=7, max=19997, vocab_size=10004
❌ ОШИБКА в Eval: индекс 19997 >= vocab_size 10004

=== ТЕСТ НА CPU ===


AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### Второй эксперимент (2 балла)

Попробуйте что-то поменять в модели или в пайплайне обучения, идеи для экспериментов можно подсмотреть выше.

In [ ]:
# Проведите второй эксперимент

### Отчет (2 балла)

Опишите проведенные эксперименты. Сравните перплексии полученных моделей. Предложите идеи по улучшению качества моделей.